# Predicting Grocery Sales:

### 1. Importing Data & Dependencies:
- my first step was to import the data
- decided to import most of the data files that I had and inspect them

In [1]:
import tensorflow as tf
import datetime, os

import matplotlib.pyplot as plt
import tensorflow as tf
import numpy as np
import pandas as pd


pd.set_option('display.max_rows', None)  # Display all rows
pd.set_option('display.max_columns', None)  # Display all columns

In [2]:
#oil imported
df_oil = pd.read_csv('./oil.csv')
df_oil.head()

,date,dcoilwtico
0,2013-01-01,NaN
1,2013-01-02,93.14
2,2013-01-03,92.97
3,2013-01-04,93.12
4,2013-01-07,93.20


In [3]:
# holidays imported
df_holidays = pd.read_csv('./holidays_events.csv')
df_holidays.head()

,date,type,locale,locale_name,description,transferred
0,2012-03-02,Holiday,Local,Manta,Fundacion de Manta,False
1,2012-04-01,Holiday,Regional,Cotopaxi,Provincializacion de Cotopaxi,False
2,2012-04-12,Holiday,Local,Cuenca,Fundacion de Cuenca,False
3,2012-04-14,Holiday,Local,Libertad,Cantonizacion de Libertad,False
4,2012-04-21,Holiday,Local,Riobamba,Cantonizacion de Riobamba,False


In [4]:
# stores imported
df_stores = pd.read_csv('./stores.csv')
df_stores.head()

,store_nbr,city,state,type,cluster
0,1,Quito,Pichincha,D,13
1,2,Quito,Pichincha,D,13
2,3,Quito,Pichincha,D,8
3,4,Quito,Pichincha,D,9
4,5,Santo Domingo,Santo Domingo de los Tsachilas,D,4


In [5]:
df_train = pd.read_csv('./train.csv')
df_train.head()

,id,date,store_nbr,family,sales,onpromotion
0,0,2013-01-01,1,AUTOMOTIVE,0.0,0
1,1,2013-01-01,1,BABY CARE,0.0,0
2,2,2013-01-01,1,BEAUTY,0.0,0
3,3,2013-01-01,1,BEVERAGES,0.0,0
4,4,2013-01-01,1,BOOKS,0.0,0


In [6]:
# transactions imported
df_transactions = pd.read_csv('./transactions.csv')
df_transactions.head()

,date,store_nbr,transactions
0,2013-01-01,25,770
1,2013-01-02,1,2111
2,2013-01-02,2,2358
3,2013-01-02,3,3487
4,2013-01-02,4,1922


### Inspecting data

- Just the head of the data set was not enough for me to understand the full data set

In [7]:
df_oil.describe()

,dcoilwtico
count,1175.000000
mean,67.714366
std,25.630476
min,26.190000
25%,46.405000
50%,53.190000
75%,95.660000
max,110.620000


In [8]:
df_holidays.describe()

,date,type,locale,locale_name,description,transferred
count,350,350,350,350,350,350
unique,312,6,3,24,103,2
top,2014-06-25,Holiday,National,Ecuador,Carnaval,False
freq,4,221,174,174,10,338


In [9]:
df_stores.describe()

,store_nbr,cluster
count,54.000000,54.000000
mean,27.500000,8.481481
std,15.732133,4.693395
min,1.000000,1.000000
25%,14.250000,4.000000
50%,27.500000,8.500000
75%,40.750000,13.000000
max,54.000000,17.000000


In [10]:
df_train.describe()

,id,store_nbr,sales,onpromotion
count,3.000888e+06,3.000888e+06,3.000888e+06,3.000888e+06
mean,1.500444e+06,2.750000e+01,3.577757e+02,2.602770e+00
std,8.662819e+05,1.558579e+01,1.101998e+03,1.221888e+01
min,0.000000e+00,1.000000e+00,0.000000e+00,0.000000e+00
25%,7.502218e+05,1.400000e+01,0.000000e+00,0.000000e+00
50%,1.500444e+06,2.750000e+01,1.100000e+01,0.000000e+00
75%,2.250665e+06,4.100000e+01,1.958473e+02,0.000000e+00
max,3.000887e+06,5.400000e+01,1.247170e+05,7.410000e+02


## 2. Preprocessing

### Checking for null values

In [11]:
print(df_train.isnull().sum())

id             0
date           0
store_nbr      0
family         0
sales          0
onpromotion    0
dtype: int64


In [12]:
print(df_transactions.isnull().sum())

date            0
store_nbr       0
transactions    0
dtype: int64


In [13]:
print(df_holidays.isnull().sum())

date           0
type           0
locale         0
locale_name    0
description    0
transferred    0
dtype: int64


In [14]:
print(df_stores.isnull().sum())

store_nbr    0
city         0
state        0
type         0
cluster      0
dtype: int64


In [15]:
print(df_oil.isnull().sum())

date           0
dcoilwtico    43
dtype: int64


### Replacing missing values with the mean:

#### Filling the missing values with the mean:


In [16]:
if "dcoilwtico" in df_oil.columns:
        df_oil['dcoilwtico'].fillna(df_oil['dcoilwtico'].mean(), inplace=True)
print(df_oil.isnull().sum())

date          0
dcoilwtico    0
dtype: int64


### Removing unnecessary columns:

#### Columns to Remove:

These are the columns that I decided to remove in order to avoid biased data:

Disregarding the entire holidays dataset may be a wrong decision; however, I believe including the data could introduce bias

Remove: onpromotion, transferred columns
remove all of the holidays
Remove: city, state, type and cluster, 
Remove id

In [17]:
'''
Dfs:
df_oil
df_transactions
df_stores
df_train
'''

print(df_oil.columns)    
print(df_stores.columns)
print(df_transactions.columns)
print(df_train.columns)

#Remove: onpromotion and id:
cols_to_drop = ['id', 'onpromotion']
df_train = df_train.drop(columns=cols_to_drop)

#Remove: whole holidays dataset
# - 

#Remove: city, state, type:
columns_to_drop = ['city', 'state','type']
df_stores = df_stores.drop(columns=columns_to_drop)


Index(['date', 'dcoilwtico'], dtype='object')
Index(['store_nbr', 'city', 'state', 'type', 'cluster'], dtype='object')
Index(['date', 'store_nbr', 'transactions'], dtype='object')
Index(['id', 'date', 'store_nbr', 'family', 'sales', 'onpromotion'], dtype='object')


In [18]:
# df_oil
# df_holidays
# df_stores
# df_transactions
# df_train

print(df_oil.columns)    
print(df_stores.columns)
print(df_transactions.columns)
print(df_train.columns)

Index(['date', 'dcoilwtico'], dtype='object')
Index(['store_nbr', 'cluster'], dtype='object')
Index(['date', 'store_nbr', 'transactions'], dtype='object')
Index(['date', 'store_nbr', 'family', 'sales'], dtype='object')


### I am not going to remove the outliers from the data sets just yet

## 3. Model Building 

### Merging data sets:

In [19]:
# Merge store and train:
df_train = pd.merge(df_train, df_stores, on='store_nbr', how='left')

# Merge transactions and train:
df_train = pd.merge(df_train, df_transactions, on=['date', 'store_nbr'], how='left')

# Merge oil prices and train:
df_train = pd.merge(df_train, df_oil, on='date', how='left')



In [20]:
print(df_train.head())

         date  store_nbr      family  sales  cluster  transactions  dcoilwtico
0  2013-01-01          1  AUTOMOTIVE    0.0       13           NaN   67.714366
1  2013-01-01          1   BABY CARE    0.0       13           NaN   67.714366
2  2013-01-01          1      BEAUTY    0.0       13           NaN   67.714366
3  2013-01-01          1   BEVERAGES    0.0       13           NaN   67.714366
4  2013-01-01          1       BOOKS    0.0       13           NaN   67.714366


In [21]:
df_train.columns

Index(['date', 'store_nbr', 'family', 'sales', 'cluster', 'transactions',
       'dcoilwtico'],
      dtype='object')

In [22]:
df_train.describe()

,store_nbr,sales,cluster,transactions,dcoilwtico
count,3.000888e+06,3.000888e+06,3.000888e+06,2.755104e+06,2.143746e+06
mean,2.750000e+01,3.577757e+02,8.481481e+00,1.694602e+03,6.791857e+01
std,1.558579e+01,1.101998e+03,4.649735e+00,9.632810e+02,2.523605e+01
min,1.000000e+00,0.000000e+00,1.000000e+00,5.000000e+00,2.619000e+01
25%,1.400000e+01,0.000000e+00,4.000000e+00,1.046000e+03,4.667000e+01
50%,2.750000e+01,1.100000e+01,8.500000e+00,1.393000e+03,5.401000e+01
75%,4.100000e+01,1.958473e+02,1.300000e+01,2.079000e+03,9.549000e+01
max,5.400000e+01,1.247170e+05,1.700000e+01,8.359000e+03,1.106200e+02


In [23]:
df_train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3000888 entries, 0 to 3000887
Data columns (total 7 columns):
 #   Column        Dtype  
---  ------        -----  
 0   date          object 
 1   store_nbr     int64  
 2   family        object 
 3   sales         float64
 4   cluster       int64  
 5   transactions  float64
 6   dcoilwtico    float64
dtypes: float64(3), int64(2), object(2)
memory usage: 160.3+ MB


In [24]:
import xgboost as xgb

#split the data into validation data and training data
from sklearn.model_selection import train_test_split

def preprocessing(df):
    #Apply one-hot encoding to the 'family' column
    df = pd.get_dummies(df, columns=['family'], prefix='family')

    #Convert 'date' column to datetime type
    df['date'] = pd.to_datetime(df['date'])

    #Extract relevant date features:
    df['year'] = df['date'].dt.year
    df['month'] = df['date'].dt.month
    df['day_of_week'] = df['date'].dt.dayofweek  # Monday=0, Sunday=6

    #Drop the original 'date' column - no longer needed
    df = df.drop(columns=['date'])
    
    return df

df_train = preprocessing(df_train)

#Features
#Target
df_train = df_train.drop(columns=['cluster'])
df_train = df_train.drop(columns=['transactions'])
df_train = df_train.drop(columns=['dcoilwtico'])
y = df_train['sales']
X = df_train.drop(columns=['sales'])

# Split the data into training and validation sets
X_train, X_valid, y_train, y_valid = train_test_split(X, y, test_size=0.2, random_state=42, shuffle=True)


#DMatrix for XGBoost training
dtrain = xgb.DMatrix(X_train, label=y_train)
dvalid = xgb.DMatrix(X_valid, label=y_valid)


#XGBoost parameters
params = {
    'objective': 'reg:squarederror',  #Regression
    'n_estimators': 100,  # Number of trees (adjust)
    'learning_rate': 0.01  # Learning rate (adjust)
}

## 4. Training the model:

In [25]:
# Train the XGBoost model
# Train the XGBoost model
model = xgb.train(params, dtrain)

# Make predictions on the validation set
y_valid_pred = model.predict(dvalid)

# Evaluate the model's performance (e.g., using Mean Absolute Error)
from sklearn.metrics import mean_absolute_error
mae = mean_absolute_error(y_valid, y_valid_pred)
print(f"Mean Absolute Error on Validation Set: {mae}")

#test it on the validation data:
#y_pred = model.predict(dvalid)

[22:32:21] WARNING: C:\buildkite-agent\builds\buildkite-windows-cpu-autoscaling-group-i-0fdc6d574b9c0d168-1\xgboost\xgboost-ci-windows\src\learner.cc:767: 
Parameters: { "n_estimators" } are not used.

Mean Absolute Error on Validation Set: 334.02932600084426


## 5. Test the model:

In [28]:
#test imported
df_test = pd.read_csv('./test.csv')
df_test.head()

,id,date,store_nbr,family,onpromotion
0,3000888,2017-08-16,1,AUTOMOTIVE,0
1,3000889,2017-08-16,1,BABY CARE,0
2,3000890,2017-08-16,1,BEAUTY,2
3,3000891,2017-08-16,1,BEVERAGES,20
4,3000892,2017-08-16,1,BOOKS,0


In [29]:
#DMatrix for to test
df_test = preprocessing(df_test)
test_id = df_test['id'].copy()
df_test = df_test.drop(columns=['id'])


#final preprocessing:
#mean values from the df_train dataset
# mean_transactions = df_train['transactions'].mean()
# mean_dcoilwtico = df_train['dcoilwtico'].mean()

# # Fill missing values in df_test with the corresponding means
# df_test['transactions'].fillna(mean_transactions, inplace=True)
# df_test['dcoilwtico'].fillna(mean_dcoilwtico, inplace=True)

df_test = df_test.drop(columns=['onpromotion'])

dtest = xgb.DMatrix(df_test)

#Use the trained XGBoost model to make predictions
y_test_pred = model.predict(dtest)


#Save to a CSV file:
df_submission = pd.DataFrame({'id': test_id, 'sales': y_test_pred})
df_submission.to_csv('sample_submission.csv', index=False)
